In [8]:
from pathlib import Path
import pandas as pd
import numpy as np

import torch
from pytorch_forecasting.data import TimeSeriesDataSet
from pytorch_forecasting.data.encoders import GroupNormalizer
from pytorch_forecasting.metrics import QuantileLoss
from pytorch_forecasting import TemporalFusionTransformer

from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint

import random

In [ ]:
DATA_PATH = Path(r"D:\C14220255\tft_dataset_full_eligible.csv")
OUT_DIR   = Path(r"D:\C14220255\outputs_tft_full")
OUT_DIR.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(
    DATA_PATH,
    parse_dates=["periode"],
    dtype={"area": str},    
    low_memory=False          
)

df = df.sort_values(["cabang","sku","periode"]).reset_index(drop=True)

print("Rows total:", len(df))
print("Periode min/max:", df["periode"].min(), "->", df["periode"].max())

req_cols = ["cabang","sku","periode","qty","is_train","is_test","sample_weight"]
miss = [c for c in req_cols if c not in df.columns]
if miss:
    raise ValueError(f"Kolom wajib hilang di dataset: {miss}")

Rows total: 159180
Periode min/max: 2021-01-01 00:00:00 -> 2024-10-01 00:00:00


In [ ]:
TRAIN_END = pd.Timestamp("2024-05-01")

if "is_train" not in df.columns or "is_test" not in df.columns:
    raise ValueError("Kolom is_train / is_test tidak ada di dataset. Cek CSV dulu.")

# pastikan urutan
df = df.sort_values(["cabang", "sku", "periode"]).reset_index(drop=True)

# hitung info seri hanya dari periode sampai TRAIN_END
base = df[df["periode"] <= TRAIN_END].copy()

grp = base.groupby(["cabang", "sku"], as_index=False)

info = grp.agg(
    n_months=("qty", "size"),
    nonzero_months=("qty", lambda s: (s > 0).sum()),
    total_qty=("qty", "sum"),
)

# zero_ratio_train dan n_train pakai flag is_train
train_part = df[df["is_train"] == 1].copy()
gtr = train_part.groupby(["cabang", "sku"])

zr = gtr["qty"].apply(lambda s: (s == 0).mean()).reset_index(name="zero_ratio_train")
ntr = gtr["qty"].count().reset_index(name="n_train")

# qty 12 bulan terakhir di train
last12 = (
    train_part.sort_values(["cabang", "sku", "periode"])
    .groupby(["cabang", "sku"], as_index=False)
    .agg(qty_12m=("qty", lambda s: s.tail(12).sum()))
)

# last non zero di train
nz = (
    train_part[train_part["qty"] > 0]
    .groupby(["cabang", "sku"], as_index=False)["periode"]
    .max()
    .rename(columns={"periode": "last_nz"})
)

info = (
    info.merge(zr, on=["cabang", "sku"], how="left")
        .merge(ntr, on=["cabang", "sku"], how="left")
        .merge(last12, on=["cabang", "sku"], how="left")
        .merge(nz, on=["cabang", "sku"], how="left")
)

# isi NA
for c in ["zero_ratio_train", "n_train", "qty_12m"]:
    info[c] = info[c].fillna(0)

info["last_nz"] = pd.to_datetime(info["last_nz"], errors="coerce")

train_end_per = TRAIN_END.to_period("M")
info["months_since_last_nz"] = 999

mask = info["last_nz"].notna()
if mask.any():
    last_nz_per = info.loc[mask, "last_nz"].dt.to_period("M")
    diff = train_end_per.ordinal - last_nz_per.astype("int64")
    info.loc[mask, "months_since_last_nz"] = diff.values

info["months_since_last_nz"] = info["months_since_last_nz"].astype(int)

# alive_recent: ada aktivitas 3 bulan terakhir
info["alive_recent"] = (
    (info["qty_12m"] > 0) &
    (info["months_since_last_nz"] <= 3)
).astype(int)


info["eligible_tft"] = (
    (info["n_months"] >= 30) &          # cukup panjang
    (info["nonzero_months"] >= 10) &    # tidak terlalu sparse
    (info["total_qty"] >= 30) &         # bukan seri sampah
    (info["zero_ratio_train"] <= 0.7) & # tidak 90% nol
    (info["alive_recent"] == 1)         # masih "hidup"
).astype(int)

SERIES_INFO_PATH = r"D:\C14220255\series_info_full_tft.csv"
info.to_csv(SERIES_INFO_PATH, index=False)
print("Saved series_info_full:", SERIES_INFO_PATH)

print("Total seri:", len(info), "| eligible_tft:", info["eligible_tft"].sum())


Saved series_info_full: D:\C14220255\series_info_full_tft.csv
Total seri: 3898 | eligible_tft: 3898


In [ ]:
DATA_PATH = r"D:\C14220255\tft_dataset_full_eligible.csv"

df = pd.read_csv(
    DATA_PATH,
    parse_dates=["periode"],
    dtype={"area": str},
    low_memory=False
)

df = df.sort_values(["cabang", "sku", "periode"]).reset_index(drop=True)

# merge eligible_tft
info = pd.read_csv(r"D:\C14220255\series_info_full_tft.csv")

df = df.merge(
    info[["cabang", "sku", "eligible_tft"]],
    on=["cabang", "sku"],
    how="left"
)

df["eligible_tft"] = df["eligible_tft"].fillna(0).astype(int)

# filter: pakai semua yang eligible (sekarang = semua)
df = df[df["eligible_tft"] == 1].copy()
df = df.sort_values(["cabang", "sku", "periode"]).reset_index(drop=True)

print("Rows after TFT filter:", len(df))
print("Unique series:", df[["cabang", "sku"]].drop_duplicates().shape[0])


Rows after TFT filter: 159180
Unique series: 3898


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd

from pytorch_forecasting.data import TimeSeriesDataSet
from pytorch_forecasting.data.encoders import GroupNormalizer
from pytorch_forecasting.metrics import QuantileLoss
from pytorch_forecasting import TemporalFusionTransformer

from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint

DATA_PATH = r"D:\C14220255\tft_dataset_full_eligible.csv"
INFO_PATH = r"D:\C14220255\series_info_full_tft.csv"
OUT_DIR   = Path(r"D:\C14220255\tft_out_full_global")
OUT_DIR.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(
    DATA_PATH,
    parse_dates=["periode"],
    dtype={"area": str},
    low_memory=False,
)

print("Rows total:", len(df))
print("Periode min/max:", df["periode"].min(), "->", df["periode"].max())

info = pd.read_csv(INFO_PATH)

df = df.merge(
    info[["cabang", "sku", "eligible_tft"]],
    on=["cabang", "sku"],
    how="left"
)

df["eligible_tft"] = df["eligible_tft"].fillna(0).astype(int)

# Hanya pakai seri eligible (sekarang 3898 semua)
df = df[df["eligible_tft"] == 1].copy()
df = df.sort_values(["cabang", "sku", "periode"]).reset_index(drop=True)

print("Rows after TFT filter:", len(df))
print("Unique series:", df[["cabang", "sku"]].drop_duplicates().shape[0])

df["qty"] = df["qty"].clip(lower=0)
df["qty_log"] = np.log1p(df["qty"])

if "spike_flag" not in df.columns:
    df["spike_flag"] = 0

df["w_tft"] = 1.0
df.loc[df["spike_flag"] == 1, "w_tft"] = 3.0  


# TIME_IDX & LAG/ROLL CLEANING

df = df.sort_values(["cabang", "sku", "periode"]).reset_index(drop=True)
df["time_idx"] = df.groupby(["cabang", "sku"]).cumcount()

rolling_cols = [
    "qty_rollmean_3","qty_rollstd_3",
    "qty_rollmean_6","qty_rollstd_6",
    "qty_rollmean_12","qty_rollstd_12",
]
rolling_cols = [c for c in rolling_cols if c in df.columns]

lag_cols = [c for c in df.columns if c.startswith("qty_lag")]

# rolling -> bfill/ffill per seri
if rolling_cols:
    df[rolling_cols] = (
        df.groupby(["cabang","sku"])[rolling_cols]
          .transform(lambda g: g.bfill().ffill())
    )

# lag -> NaN jadi 0
if lag_cols:
    df[lag_cols] = df[lag_cols].fillna(0)

for col in ["event_flag_lag1","holiday_count_lag1","rainfall_lag1"]:
    if col in df.columns:
        df[col] = df[col].fillna(0)

print("Sisa NA setelah cleaning:")
print(df.isna().sum()[df.isna().sum() > 0])


Rows total: 159180
Periode min/max: 2021-01-01 00:00:00 -> 2024-10-01 00:00:00
Rows after TFT filter: 159180
Unique series: 3898
Sisa NA setelah cleaning:
Series([], dtype: int64)


In [ ]:
from pytorch_forecasting.data import TimeSeriesDataSet
from pytorch_forecasting.data.encoders import GroupNormalizer

# TRAIN SPLIT
train_df = df[df["is_train"] == 1].copy()

print("Baris train:", len(train_df))
print("Range time_idx train:", train_df["time_idx"].min(), "->", train_df["time_idx"].max())

# 6 bulan terakhir sebagai validasi internal
training_cutoff = train_df["time_idx"].max() - 6
print("training_cutoff:", training_cutoff)

# KONFIG DATASET
static_cat = ["cabang", "sku"]

known_reals = [
    "time_idx",
    "event_flag", "event_flag_lag1",
    "holiday_count", "holiday_count_lag1",
    "rainfall_lag1",
    "spike_flag",
]
known_reals = [c for c in known_reals if c in df.columns]

unknown_reals = ["qty_log"] + rolling_cols + lag_cols

min_encoder_length = 12
max_encoder_length = 24
max_prediction_length = 1

training_ds = TimeSeriesDataSet(
    train_df,
    time_idx="time_idx",
    target="qty_log",
    group_ids=["cabang", "sku"],
    weight="w_tft",

    min_encoder_length=min_encoder_length,
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,

    static_categoricals=static_cat,
    static_reals=[],
    time_varying_known_categoricals=[],
    time_varying_known_reals=known_reals,
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=unknown_reals,

    target_normalizer=GroupNormalizer(groups=["cabang","sku"]),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,

    # ini yang bikin split train/val internal
    min_prediction_idx=training_cutoff + 1,
)

batch_size = 64

train_loader = training_ds.to_dataloader(
    train=True,
    batch_size=batch_size,
    num_workers=0,
)

val_loader = training_ds.to_dataloader(
    train=False,
    batch_size=batch_size,
    num_workers=0,
)

print("Jumlah sample train:", len(train_loader.dataset))
print("Jumlah sample val  :", len(val_loader.dataset))


Baris train: 159135
Range time_idx train: 0 -> 40
training_cutoff: 34
Jumlah sample train: 69481
Jumlah sample val  : 69481


In [14]:
# cek berapa seri yg punya is_test == 1
test_series = (
    df[df["is_test"] == 1]
      .groupby(["cabang","sku"])
      .size()
      .reset_index(name="n_test")
)

print("Jumlah seri yang punya periode test:", len(test_series))
print(test_series.head(10))


Jumlah seri yang punya periode test: 9
  cabang          sku  n_test
0    02A   BUVW001KSW       5
1    05A   BUVW001KSW       5
2    13A  DOPQ001K002       5
3    13I   BUVW001KSW       5
4    14A   BUVW001KSW       5
5    16C  DOPQ001K009       5
6    17A  DOPQ001K002       5
7    23A   BUVW001KSW       5
8    29A   BUVW001KSW       5


In [ ]:
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.callbacks import EarlyStopping
from pytorch_forecasting import TemporalFusionTransformer
from pytorch_forecasting.metrics import QuantileLoss
import random

seed_everything(42)

loss_q = QuantileLoss(quantiles=[0.5])

def sample_params_rs():
    return {
        "hidden_size": random.choice([16, 24, 32, 40]),
        "lstm_layers": random.choice([1, 2]),
        "dropout": random.uniform(0.1, 0.3),
        "attention_head_size": random.choice([1, 2, 4]),
        "learning_rate": random.uniform(3e-4, 3e-3),
    }

N_RS = 12     

rs_results = []

for i in range(N_RS):
    p = sample_params_rs()
    print(f"\n=== RS Trial {i+1}/{N_RS} ===")
    print(p)

    model = TemporalFusionTransformer.from_dataset(
        training_ds,
        hidden_size=p["hidden_size"],
        lstm_layers=p["lstm_layers"],
        dropout=p["dropout"],
        attention_head_size=p["attention_head_size"],
        learning_rate=p["learning_rate"],
        loss=loss_q,
        output_size=1,
    )

    trainer = Trainer(
        max_epochs=20,
        accelerator="cpu",
        callbacks=[EarlyStopping(monitor="val_loss", patience=3)],
        enable_progress_bar=True
    )

    trainer.fit(model, train_loader, val_loader)
    val_loss = trainer.callback_metrics["val_loss"].item()
    
    rs_results.append((val_loss, p))
    print("val_loss =", val_loss)

best_rs = sorted(rs_results, key=lambda x: x[0])[0][1]
print("\nBest RS Params:", best_rs)


Seed set to 42
d:\C14220255\.venv\Lib\site-packages\lightning\pytorch\utilities\parsing.py:210: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
d:\C14220255\.venv\Lib\site-packages\lightning\pytorch\utilities\parsing.py:210: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.



=== RS Trial 1/12 ===
{'hidden_size': 16, 'lstm_layers': 1, 'dropout': 0.24831009995196657, 'attention_head_size': 1, 'learning_rate': 0.0009026689930018215}


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
d:\C14220255\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

   | Name                               | Type                            | Params | Mode 
------------------------------------------------------------------------------------------------
0  | loss 

d:\C14220255\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.
d:\C14220255\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Epoch 19: 100%|██████████| 1085/1085 [04:15<00:00,  4.24it/s, v_num=52, train_loss_step=0.315, val_loss=0.274, train_loss_epoch=0.304]

...

Epoch 19: 100%|██████████| 1085/1085 [05:05<00:00,  3.56it/s, v_num=62, train_loss_step=0.239, val_loss=0.190, train_loss_epoch=0.217]
val_loss = 0.19012488424777985

=== RS Trial 12/12 ===
{'hidden_size': 32, 'lstm_layers': 1, 'dropout': 0.22182620113339763, 'attention_head_size': 1, 'learning_rate': 0.0017421776679802116}


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

   | Name                               | Type                            | Params | Mode 
------------------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0      | train
1  | logging_metrics                    | ModuleList                      | 0      | train
2  | input_embeddings                   | MultiEmbedding                  | 33.2 K | train
3  | prescalers                         | ModuleDict                      | 480    | train
4  | static_variable_selection          | VariableSelectionNetwork        | 3.3 K  | train
5  | encoder_variable_selection         | VariableSelectionNetwork        | 

Epoch 19: 100%|██████████| 1085/1085 [04:28<00:00,  4.04it/s, v_num=63, train_loss_step=0.209, val_loss=0.234, train_loss_epoch=0.257]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 1085/1085 [04:28<00:00,  4.04it/s, v_num=63, train_loss_step=0.209, val_loss=0.234, train_loss_epoch=0.257]
val_loss = 0.2343195378780365

Best RS Params: {'hidden_size': 40, 'lstm_layers': 2, 'dropout': 0.13253081943121697, 'attention_head_size': 2, 'learning_rate': 0.0008656689830830167}


In [16]:
import optuna
from lightning.pytorch.callbacks import EarlyStopping
from pytorch_forecasting import TemporalFusionTransformer

def objective(trial):
    p = {
        "hidden_size"        : trial.suggest_int("hidden_size", best_rs["hidden_size"]-8, best_rs["hidden_size"]+8),
        "lstm_layers"        : trial.suggest_int("lstm_layers", 1, 2),
        "dropout"            : trial.suggest_float("dropout", 0.05, 0.25),
        "attention_head_size": trial.suggest_categorical("attention_head_size", [1, 2, 4]),
        "learning_rate"      : trial.suggest_float("learning_rate", 1e-4, 2e-3, log=True),
    }

    model = TemporalFusionTransformer.from_dataset(
        training_ds,
        hidden_size=p["hidden_size"],
        lstm_layers=p["lstm_layers"],
        dropout=p["dropout"],
        attention_head_size=p["attention_head_size"],
        learning_rate=p["learning_rate"],
        loss=loss_q,
        output_size=1,
    )

    trainer = Trainer(
        max_epochs=25,
        accelerator="cpu",
        callbacks=[EarlyStopping(monitor="val_loss", patience=4)],
        enable_progress_bar=False
    )

    trainer.fit(model, train_loader, val_loader)

    return trainer.callback_metrics["val_loss"].item()


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=25)

final_params = study.best_params
print("\n=== BEST OPTUNA PARAMS ===")
print(final_params)


[I 2025-11-16 16:59:51,466] A new study created in memory with name: no-name-6c175c2f-c0fb-4d38-81db-a4cafe42281d
d:\C14220255\.venv\Lib\site-packages\lightning\pytorch\utilities\parsing.py:210: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
d:\C14220255\.venv\Lib\site-packages\lightning\pytorch\utilities\parsing.py:210: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

   | Name                               | Type                         


=== BEST OPTUNA PARAMS ===
{'hidden_size': 40, 'lstm_layers': 1, 'dropout': 0.05698786555311375, 'attention_head_size': 4, 'learning_rate': 0.0017647841976014306}


In [17]:
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping

checkpoint_cb = ModelCheckpoint(
    dirpath="tft_checkpoints_full",
    filename="tft_final",
    monitor="val_loss",
    save_top_k=1,
    mode="min"
)

early_cb = EarlyStopping(monitor="val_loss", patience=6, mode="min")

final_model = TemporalFusionTransformer.from_dataset(
    training_ds,
    hidden_size=final_params["hidden_size"],
    lstm_layers=final_params["lstm_layers"],
    dropout=final_params["dropout"],
    attention_head_size=final_params["attention_head_size"],
    learning_rate=final_params["learning_rate"],
    loss=loss_q,
    output_size=1,
)

trainer = Trainer(
    max_epochs=60,
    accelerator="cpu",
    callbacks=[checkpoint_cb, early_cb],
    enable_progress_bar=True
)

trainer.fit(final_model, train_loader, val_loader)

print("Best checkpoint:", checkpoint_cb.best_model_path)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores

   | Name                               | Type                            | Params | Mode 
------------------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0      | train
1  | logging_metrics                    | ModuleList                      | 0      | train
2  | input_embeddings                   | MultiEmbedding                  | 41.5 K | train
3  | prescalers                         | ModuleDict                      | 480    | train
4  | static_variable_selection          | VariableSelectionNetwork        | 3.9 K  | train
5  | encoder_variable_selection         | VariableSelectionNetwork        | 38.5 K | train
6  | decoder_variable_selection         | VariableSelectionNetwork        | 9.7 K  | train
7  | static_context_variable_selection  | GatedResidualNetwork            | 6.6 K  | train
8  | sta

Epoch 53: 100%|██████████| 1085/1085 [04:58<00:00,  3.64it/s, v_num=89, train_loss_step=0.123, val_loss=0.160, train_loss_epoch=0.173] 
Best checkpoint: D:\C14220255\tft_checkpoints_full\tft_final.ckpt


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
from pytorch_forecasting.data import TimeSeriesDataSet


PROJECT_ROOT = Path(r"D:\C14220255")  
OUT_DIR = PROJECT_ROOT / "tft_full_eval"
OUT_DIR.mkdir(parents=True, exist_ok=True)


# 4.1 Helper metric di level qty (bukan log)
def calc_metrics_level(dfm: pd.DataFrame):
    dfm = dfm.dropna(subset=["qty", "qty_pred"]).copy()

    err = dfm["qty_pred"] - dfm["qty"]

    mse  = float(np.mean(err**2))
    rmse = float(np.sqrt(mse))
    mae  = float(np.mean(np.abs(err)))

    nonzero = dfm["qty"] != 0
    if nonzero.sum() > 0:
        mape = float(
            np.mean(
                np.abs(dfm.loc[nonzero, "qty_pred"] - dfm.loc[nonzero, "qty"])
                / dfm.loc[nonzero, "qty"]
            ) * 100
        )
    else:
        mape = np.nan

    smape = float(
        np.mean(
            2 * np.abs(dfm["qty_pred"] - dfm["qty"])
            / (np.abs(dfm["qty_pred"]) + np.abs(dfm["qty"]) + 1e-9)
        ) * 100
    )

    return rmse, mae, mape, smape, mse

# 4.2 Prediksi TRAIN (pakai training_ds)
#     output di log, balik ke level dengan expm1
train_raw = final_model.predict(training_ds, return_index=True)
y_log_pred_train = train_raw.output.squeeze(-1)
idx_train = train_raw.index

df_train_pred = pd.DataFrame({
    "cabang":   idx_train["cabang"],
    "sku":      idx_train["sku"],
    "time_idx": idx_train["time_idx"],
    "qty_log_pred": y_log_pred_train,
})

# merge dengan info asli (qty, periode, is_train)
df_train_pred = df_train_pred.merge(
    df[["cabang","sku","time_idx","qty","periode","is_train","is_test"]],
    on=["cabang","sku","time_idx"],
    how="left"
)

# collapse duplikat window -> rata-rata per (seri, time_idx, periode)
df_train_pred = (
    df_train_pred
    .groupby(["cabang","sku","time_idx","periode"], as_index=False)
    .agg(
        qty_log_pred=("qty_log_pred", "mean"),
        qty=("qty", "first"),
        is_train=("is_train", "max"),
        is_test=("is_test", "max"),
    )
)

# balik ke level
df_train_pred["qty_pred"] = np.expm1(df_train_pred["qty_log_pred"])

# keep hanya baris train yang beneran train
df_train_pred = df_train_pred[df_train_pred["is_train"] == 1].copy()

print("Train pred shape:", df_train_pred.shape)

# 4.3 Prediksi FULL (train+val+test) dari full df
full_ds = TimeSeriesDataSet.from_dataset(
    training_ds,
    df,
    stop_randomization=True
)

full_raw = final_model.predict(full_ds, return_index=True)
y_log_pred_full = full_raw.output.squeeze(-1)
idx_full = full_raw.index

df_pred_all = pd.DataFrame({
    "cabang":   idx_full["cabang"],
    "sku":      idx_full["sku"],
    "time_idx": idx_full["time_idx"],
    "qty_log_pred": y_log_pred_full,
})

df_pred_all = df_pred_all.merge(
    df[["cabang","sku","time_idx","qty","periode","is_train","is_test"]],
    on=["cabang","sku","time_idx"],
    how="left"
)

# collapse window duplicate
df_pred_all = (
    df_pred_all
    .groupby(["cabang","sku","time_idx","periode"], as_index=False)
    .agg(
        qty_log_pred=("qty_log_pred", "mean"),
        qty=("qty", "first"),
        is_train=("is_train", "max"),
        is_test=("is_test", "max"),
    )
)

df_pred_all["qty_pred"] = np.expm1(df_pred_all["qty_log_pred"])

print("Full pred shape:", df_pred_all.shape)

# 4.4 Filter hanya periode test (Jun–Okt 2024)
EVAL_START = pd.Timestamp("2024-06-01")
EVAL_END   = pd.Timestamp("2024-10-01")

df_test_pred = df_pred_all[
    (df_pred_all["is_test"] == 1) &
    (df_pred_all["periode"] >= EVAL_START) &
    (df_pred_all["periode"] <= EVAL_END)
].copy()

print("Test pred shape:", df_test_pred.shape)
print("\nJumlah seri yang punya periode test:")
print(
    df_test_pred.groupby(["cabang","sku"])["periode"]
                .nunique()
                .reset_index(name="n_test")
)


# 4.5 Metric GLOBAL train vs test
rmse_tr, mae_tr, mape_tr, smape_tr, mse_tr = calc_metrics_level(df_train_pred)
rmse_te, mae_te, mape_te, smape_te, mse_te = calc_metrics_level(df_test_pred)

summary_metrics = pd.DataFrame([{
    "RMSE_train": rmse_tr,
    "MAE_train": mae_tr,
    "MAPE%_train": mape_tr,
    "sMAPE%_train": smape_tr,
    "MSE_train": mse_tr,
    "RMSE_test": rmse_te,
    "MAE_test": mae_te,
    "MAPE%_test": mape_te,
    "sMAPE%_test": smape_te,
    "MSE_test": mse_te,
}])

print("\nGlobal metrics:")
print(summary_metrics)


# 4.6 Metric per seri (hanya seri yang punya test)
rows = []
for (cab, sku), g in df_test_pred.groupby(["cabang","sku"], sort=False):
    rmse, mae, mape, smape, mse = calc_metrics_level(g)
    rows.append({
        "cabang": cab,
        "sku": sku,
        "n_test": len(g),
        "MSE_test": mse,
        "RMSE_test": rmse,
        "MAE_test": mae,
        "MAPE%_test": mape,
        "sMAPE%_test": smape,
    })

metrics_by_series = pd.DataFrame(rows).sort_values(["cabang","sku"]).reset_index(drop=True)
print("\nMetrics per seri (test):")
print(metrics_by_series)

# 4.7 Save semua ke CSV
train_path   = OUT_DIR / "tft_full_train_predictions.csv"
test_path    = OUT_DIR / "tft_full_test_predictions.csv"
summary_path = OUT_DIR / "tft_full_metrics_summary.csv"
series_path  = OUT_DIR / "tft_full_metrics_by_series.csv"

df_train_pred.to_csv(train_path, index=False)
df_test_pred.to_csv(test_path, index=False)
summary_metrics.to_csv(summary_path, index=False)
metrics_by_series.to_csv(series_path, index=False)

print("\nSaved:")
print(" -", train_path)
print(" -", test_path)
print(" -", summary_path)
print(" -", series_path)


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
d:\C14220255\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Train pred shape: (22705, 9)


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
d:\C14220255\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Full pred shape: (22750, 9)
Test pred shape: (45, 9)

Jumlah seri yang punya periode test:
  cabang          sku  n_test
0    02A   BUVW001KSW       5
1    05A   BUVW001KSW       5
2    13A  DOPQ001K002       5
3    13I   BUVW001KSW       5
4    14A   BUVW001KSW       5
5    16C  DOPQ001K009       5
6    17A  DOPQ001K002       5
7    23A   BUVW001KSW       5
8    29A   BUVW001KSW       5

Global metrics:
   RMSE_train  MAE_train  MAPE%_train  sMAPE%_train     MSE_train  \
0  105.728947   22.74733    38.105686     58.933578  11178.610282   

     RMSE_test     MAE_test  MAPE%_test  sMAPE%_test      MSE_test  
0  2169.523608  1368.443838   41.529732    37.248821  4.706833e+06  

Metrics per seri (test):
  cabang          sku  n_test      MSE_test    RMSE_test     MAE_test  \
0    02A   BUVW001KSW       5  5.025600e+06  2241.784921  1943.498682   
1    05A   BUVW001KSW       5  4.803083e+05   693.042748   459.134521   
2    13A  DOPQ001K002       5  3.821433e+06  1954.848594  1632.913818 

In [20]:
import matplotlib.pyplot as plt

PLOT_DIR = OUT_DIR / "plots_test_series"
PLOT_DIR.mkdir(parents=True, exist_ok=True)

# pastikan df_test_pred dari blok sebelumnya ada
print("Jumlah seri test:", df_test_pred[["cabang","sku"]].drop_duplicates().shape[0])

for (cab, sku), g in df_test_pred.groupby(["cabang","sku"], sort=False):
    g = g.sort_values("periode").copy()

    plt.figure(figsize=(6, 4))
    plt.plot(g["periode"], g["qty"], marker="o", label="Actual")
    plt.plot(g["periode"], g["qty_pred"], marker="o", label="Predicted")

    plt.title(f"{cab} - {sku}\nPeriode test (Jun–Okt 2024)")
    plt.xlabel("Periode")
    plt.ylabel("Qty")
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()

    fname = f"{cab}_{sku}_test_actual_vs_pred.png"
    plt.savefig(PLOT_DIR / fname, dpi=150)
    plt.close()

print("Plot disimpan di folder:", PLOT_DIR)


Jumlah seri test: 9
Plot disimpan di folder: D:\C14220255\tft_full_eval\plots_test_series
